# GeekBrains, Recommendation Systems
# Lesson 3 Homework

**Импорт библиотек**

In [1]:
import numpy as np

## Задание 1
**Ситуация**: Вы работаете Data Scientist'ом в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи.

**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail.

**Ожидание:** Отправляем e-mail с топ-10 товарами, отсортированными по вероятности.

**Реальность:**
1. Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
2. По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
3. А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
4. Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
5. Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
6. Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
7. Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
8. Cколько мы хотим зарабатывать с одного привлеченного юзера?

**Попытаться ответить на вопросы/выдвинуть гипотезы:**

9. А точно нужно сортировать по вероятности?
10. Какую метрику использовать?
11. Сколько раз в неделю отправляем рассылку?
12. В какое время отправляем рассылку?
13. Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
14. Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

### Решение Задания 1

Если крупный продуктовый ритейлер, то значит торговля в большой степени продуктами питания, потребители покупают достаточно часто.

9. А точно нужно сортировать по вероятности?

Стоит сортировать не по вероятности, а по потенциальной выручке с товара - выручка с товара * вероятность.

10. Какую метрику использовать?

Money precision - она учитывает то, что мы рекомендуем именно релевантные товары, а также стоимость каждого рекомендованного товара.

11. Сколько раз в неделю отправляем рассылку?

2 раза в неделю вполне достаточно: 1. В пятницу, перед большой закупкой продуктов на выходных, 2. В середине недели, примерно во вторник, для подогрева интереса и покупки более мелких, импульсивных товаров.

12. В какое время отправляем рассылку?

В пятницу - утром или днем, чтобы увидели перед походом в магазин, а если пользователь заказывает онлайн - вечером пятницы или днем субботы. Во вторник - примерно за 1-2 часа до конца рабочего дня.

13. Будем отправлять одному юзеру много раз наши рекомендации. Как добиться того, чтобы они хоть немного отличались?

Можно рекомендовать не самый топ-10 товаров, а набирать 10 товаров из топ-5 или топ-7 товаров наиболее интересующих пользователя категорий. Таким образом, будет некий запас товаров, благодаря которым можно варьировать рекомендации. Еще один способ - дать некоторым категориям товаров увеличенный вес в соответствии со днем недели рассылки. Например, рекомендовать больше сладостей и закусок в середине недели, а хозяйственные товары оставить на конец недели.

14. Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

Да, не стоит рекомендовать 5 различных брендов молока. "Разные" товары - те, которые при покупке пользователем первого товара не сильно влияют на покупку второго товара. Пример - вряд ли пользователь купит 2 разных вида молока, он просто возьмет 1 из них, другая рекомендация окажется бесполезной в повышении суммы продаж. Вместо этой бесполезной рекомендации стоит показать товар другой группы - хлеб, к примеру. Добить "разности" товаров можно с помощью ограничения на вхождение в рекомендации товаров каждой категории (напр. макс 1-2 молочных товара). Также можно найти "похожие" товары с помощью близости их векторов - если она выше определенного порога, то не рекомендуем эти товары вместе. Еще один вариант - посмотреть в датасете, какие товары никогда или очень редко покупаются вместе - они могут быть слишком похожи. Последний метод можно также объединить с методом подсчета близости векторов товаров.

## Задание 2
Доделать прошлые домашния задания.

### Решение Задания 2

+

## Задание 3
Прочитать статьи BM25/MatrixFactorization:
- BM25 - https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.
- Matrix factorization (ALS, SVD) - https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/

### Решение Задания 3

+

## Задание 4
Поэкспериментировать с ALS (grid-search).

### Решение Задания 4